***
## Обработка входящих сообщений

Сообщения в Telegram делятся на несколько типов: 

* простые текстовые сообщения,

* сообщения-картинки,

* ещё есть стикеры, файлы, аудио и прочие.

Каждый тип можно обрабатывать по-разному: например, текстовые сообщения можно анализировать и отвечать на них текстовым сообщением, полученные картинки и файлы сохранять или пересылать, а на стикеры отвечать стикерами. 

Для обработки каждого типа сообщений используется свой хендлер (от англ. handler, «обработчик»). Это общеупотребимый в программировании термин, он обозначает функцию или другой вызываемый объект, который срабатывает при наступлении какого-то события. 

Чтобы превратить обычную функцию в хендлер, в pyTelegramBotAPI применяется декоратор `@bot.message_handler`, где `bot` — это объект бота, экземпляр класса `TeleBot`. 

В коде программы-бота может быть несколько хендлеров. Каждое полученное сообщение анализируется — и вызывается хендлер, наиболее подходящий для конкретного сообщения.

Для начала научим бота отвечать на простые текстовые сообщения.

***
## Обработка входящих текстовых сообщений

Чтобы бот мог обработать входящие сообщения, первым делом понадобится функция-обработчик. Объявим её:


In [ ]:
# kittybot/kittybot.py
from telebot import TeleBot  # Импортируем класс.


bot = TeleBot(token='<token>')  # Создаём объект бота с токеном.

def say_hi():
    ...  # Здесь будет логика обработки полученного сообщения,
    ...  # создание и отправка ответа. 

Чтобы функция `say_hi()` стала обработчиком сообщений, обернём её декоратором `@bot.message_handler`.

Декоратор `@bot.message_handler` передаёт в декорируемую функцию аргумент `message` — объект, в котором хранятся все данные о полученном сообщении: текст сообщения, ID отправителя и много другой полезной информации.

>💡 Полное описание объекта `message` [есть в документации](https://pytba.readthedocs.io/ru/latest/types.html#telebot.types.Message). Этот объект знает о сообщении всё!

***
## Параметры декоратора @bot.message_handler

Бот может содержать несколько разных обработчиков, у каждого из которых будет своя специализация: один будет реагировать лишь на текстовые сообщения, другой — обрабатывать сообщения с картинками, третий — работать только с голосовыми сообщениями. 

Настройки «специализации» обработчика устанавливаются в параметрах декоратора `@bot.message_handler`; вот некоторые из них:

* `content_types=[список_типов]`: типы сообщений, для которых будет вызван обработчик. Значение параметра по умолчанию: `['text']`.

    **Пример**: `@bot.message_handler(content_types=['text', 'voice'])`. 

    Такой обработчик будет вызван для текстовых и голосовых сообщений, а остальные проигнорирует.

* `commands=[список_команд]`: обработчик с таким параметром будет вызван только для команд — предустановленных разработчиком сообщений, начинающихся с символа `/`. Самые распространённые команды — это `/start`, `/help`, `/about`, но разработчик бота может придумать и применить любые другие команды, например `/byebye` для выхода из чата.

    **Пример**: `@bot.message_handler(commands=['start', 'byebye'])`. 

    Такой обработчик среагирует исключительно на перечисленные команды (слеш в значениях параметра `commands` указывать не нужно).

* `func=функция`: обработчик будет вызван, если функция, переданная в параметр `func`, вернула `True`. В параметр `func` можно передать и lambda-функцию, и обычную функцию. Любая из таких функций обязательно должна ожидать на вход объект сообщения `message`; функция может проанализировать этот объект и решить, вызывать ли обработчик.

    **Пример**: 


In [ ]:
@bot.message_handler(func=lambda message: message.chat.id == 1001)
def answer_to_boss(message):
    ...

   
Можно сделать то же самое и без lambda-функции:


In [ ]:
def check_boss_id(message):
    return message.chat.id == 1001

# В аргумент func передаём обычную функцию.
@bot.message_handler(func=check_boss_id)
def answer_to_boss(message):
    ...


В lambda-функцию (или в обычную функцию) передаётся объект сообщения `message`; функция проверяет идентификатор чата, из которого пришло сообщение. Если ID чата равен 1001, функция вернёт `True` — и обработчик будет вызван.

Таким образом, в коде может быть несколько функций с декоратором `@bot.message_handler`, и у декораторов будут разные параметры.


In [ ]:
...

@bot.message_handler(commands=['start'])
def wake_up(message):
    ...

@bot.message_handler(func=lambda message: message.chat.id == 1001)
def answer_to_boss(message):  # Если chat.id с тимлидом — 1001, для него
    ...                       # можно настроить особый обработчик.

@bot.message_handler(content_types=['voice'])
def voice_message(message):
    ... 

Документация [расскажет подробнее об этом декораторе и о других его параметрах](https://pytba.readthedocs.io/ru/latest/sync_version/index.html#telebot.TeleBot.message_handler).

В общем случае обработчики описываются так:

1. Создаётся функция, которая содержит всю логику обработки входящего сообщения.
Функция должна принимать аргумент `message`.

2. К функции применяется декоратор `@bot.message_handler` — так функция назначается **обработчиком сообщений**. При необходимости в декоратор передаются параметры, определяющие, для каких сообщений должен вызываться обработчик.

Настроим бота: напишем код, который в ответ на любое текстовое сообщение будет отправлять приветственную фразу.

Удалите из файла *kittybot/kittybot.py* прежний код и добавьте новый:


In [ ]:
# kittybot/kittybot.py
from telebot import TeleBot


bot = TeleBot(token='<token>')

@bot.message_handler(content_types=['text'])
def say_hi(message):
    # Из объекта message получаем данные о чате, из которого пришло сообщение,
    # и сохраняем эти данные в переменную chat.
    chat = message.chat
    # Получаем id чата:
    chat_id = chat.id
    # В ответ на любое текстовое сообщение в тот же чат
    # будет отправлено сообщение 'Привет, я KittyBot!'
    bot.send_message(chat_id=chat_id, text='Привет, я KittyBot!') 

Обработчик готов. Но чтобы обработать сообщение, сначала надо его получить. 

***
## Регулярные запросы к серверу: поллинг

Чтобы узнать, нет ли новых сообщений на сервере, объект бота может применить **поллинг** — отправку регулярных запросов к API Telegram. Бот начнёт периодически опрашивать сервер: «Мне кто-нибудь написал? А теперь? А сейчас? Ну что, уже написал кто?» Когда кто-нибудь напишет боту, это сообщение появится на сервере. Оно будет отправлено в ответе на очередной запрос, затем передано в программу — и станет доступно для обработки. 

Для запуска и выполнения поллинга у класса `TeleBot` есть метод `polling()`. Инструкция `bot.polling()` запустит регулярный опрос сервера.

Периодичность опроса можно менять, передав методу `polling()` именованный параметр `interval` и указав нужный интервал запросов (в секундах, `int`). Если значение равно 0 (по умолчанию это так), бот будет немедленно посылать новый запрос после получения ответа на предыдущий.


In [ ]:
bot.polling(interval=20) 

При увеличении интервала замедлится реакция бота: он будет реагировать на сообщения пользователя с задержкой.

Добавим запуск поллинга в код бота; интервал оставим по умолчанию:


In [ ]:
# kittybot/kittybot.py
from telebot import TeleBot


bot = TeleBot(token='<token>')

@bot.message_handler(content_types=['text'])
def say_hi(message):
    chat = message.chat
    chat_id = chat.id
    bot.send_message(chat_id=chat_id, text='Привет, я KittyBot!')


# Метод polling() запускает процесс; приложение начнёт
# отправлять регулярные запросы для получения входящих сообщений.
bot.polling() 

Запустите программу. После запуска программа не остановится до тех пор, пока её выполнение не будет прервано комбинацией клавиш Ctrl + C или не будет закрыта консоль.

Механизм поллинга будет работать в ожидании новых сообщений. Отправьте своему боту любое текстовое сообщение. Если всё сделано правильно, вы получите ответ.



***
## Порядок обработчиков в коде

Когда бот получает сообщение, он подбирает для него подходящий обработчик, читая код сверху вниз. Для обработки полученного сообщения будет применён первый же подходящий хендлер, который будет обнаружен в коде. И тут возможны конфликты и путаница.

Поэкспериментируем: добавим в код обработчик для команды /start и проверим, как он сработает.

Для инициализации бота пользователь должен отправить ему команду /start. При получении этой команды бот инициализируется и «получит право» общаться с пользователем. Реакцию бота на эту команду можно расширить. Настроим бота так, чтобы при инициализации он отправлял пользователю сообщение «Спасибо, что включили меня». 

Добавьте в код функцию-обработчик `wake_up()`, бот должен вызвать её при получении команды /start:


In [ ]:
# kittybot/kittybot.py
from telebot import TeleBot


bot = TeleBot(token='<token>')

@bot.message_handler(content_types=['text'])
def say_hi(message):
    chat = message.chat
    chat_id = chat.id
    bot.send_message(chat_id=chat_id, text='Привет, я KittyBot!')

@bot.message_handler(commands=['start'])
def wake_up(message):
    # В ответ на команду /start
    # должно быть отправлено сообщение "Спасибо, что включили меня".
    chat = message.chat
    chat_id = chat.id
    bot.send_message(chat_id=chat_id, text='Спасибо, что включили меня')

bot.polling() 

Запустите программу и отправьте боту команду /start.

Вместо сообщения «Спасибо, что включили меня» бот отправил «Привет, я KittyBot!». Что-то пошло не так.

Бот воспринимает команды как обычные текстовые сообщения, а обработчик для всех текстовых сообщений `say_hi()` стоит в коде выше, чем `wake_up()`. Бот применил первый же подходящий обработчик для текстовых сообщений, который нашёл, — и всё пошло не по плану.

Чтобы всё работало как задумано, `wake_up()` должен стоять в коде выше, чем обработчик для текстовых сообщений `say_hi()`.

>💡 Общий принцип таков: узкоспециализированные обработчики в коде должны располагаться выше, чем обработчики «общего назначения».

При правильном порядке обработчиков (сначала `wake_up()`, а потом `say_hi()`) KittyBot будет реагировать на текстовые сообщения фразой «Привет, я KittyBot!», а при получении команды `/start` поблагодарит за то, что его включили. Сама галантность.